In [77]:
!pip install -U google-generativeai

In [79]:
import pandas as pd
import numpy as np
import json
import os
from tqdm.auto import tqdm
import google.generativeai as genai
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [81]:
df=pd.read_csv(r"C:\Users\adity\Fynd_AI_Intern_Assignment\data\yelp.csv")
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [83]:
df=df[["text", "stars"]]
df

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5
...,...,...
9995,First visit...Had lunch here today - used my G...,3
9996,Should be called house of deliciousness!\n\nI ...,4
9997,I recently visited Olive and Ivy for business ...,4
9998,My nephew just moved to Scottsdale recently so...,2


In [85]:
df_sample = df.sample(n=200, random_state=42)
df_sample

,text,stars
6252,We got here around midnight last Friday... the...,4
4684,Brought a friend from Louisiana here. She say...,5
1731,"Every friday, my dad and I eat here. We order ...",3
4742,"My husband and I were really, really disappoin...",1
4521,Love this place! Was in phoenix 3 weeks for w...,5
...,...,...
4516,"Well, this is where I got my iPad, and I am ho...",4
1261,I finally made it to this place after hearing ...,4
3160,The best example North Scottsdale has of tradi...,3
970,Very good food! We ate some shrimp tacos and m...,4


In [87]:
df_sample = df.sample(n=200, random_state=42).reset_index(drop=True)


In [89]:
df_sample

,text,stars
0,We got here around midnight last Friday... the...,4
1,Brought a friend from Louisiana here. She say...,5
2,"Every friday, my dad and I eat here. We order ...",3
3,"My husband and I were really, really disappoin...",1
4,Love this place! Was in phoenix 3 weeks for w...,5
...,...,...
195,"Well, this is where I got my iPad, and I am ho...",4
196,I finally made it to this place after hearing ...,4
197,The best example North Scottsdale has of tradi...,3
198,Very good food! We ate some shrimp tacos and m...,4


In [170]:
from getpass import getpass
os.environ['GEMINI_API_KEY'] = getpass("Enter your API key:")

Enter your API key: ········


In [203]:
model = genai.GenerativeModel("gemini-2.0-flash")

## Approach-1 : Zero-Shot Prompt

In [206]:
prompt_zero_shot = """
You rate Yelp restaurant reviews from 1–5 stars.

Return ONLY valid JSON with:
- predicted_stars: integer 1–5
- explanation: short reason

Review:
"{review_text}"
"""

## Approach-2 : Format-Strict JSON

In [209]:
prompt_format_strict = """
You are a STRICT JSON generator. No extra words. No markdown.

Task:
Read the Yelp review and output ONLY this JSON format:

{{
  "predicted_stars": 4,
  "explanation": "Example explanation here."
}}

Now process the following review:
"{review_text}"
"""

## Approach-3 : Chain-of-Thought (CoT) Prompt

In [212]:
prompt_cot = """
Classify Yelp reviews using short reasoning.

Steps:
1. Briefly decide sentiment: very negative, negative, neutral, positive, very positive.
2. Convert sentiment to a rating 1–5.
3. Output ONLY JSON:
{{
  "predicted_stars": "integer 1-5",
  "explanation": "one short sentence"
}}

Review:
"{review_text}"
"""


In [214]:
prompt = current_prompt_template.replace("{review_text}", review_text)


In [216]:
all_predictions = []
all_actual = []

for idx in tqdm(range(len(df_sample))):
    review_text = df_sample.loc[idx, "text"]
    actual_star = int(df_sample.loc[idx, "stars"])

    # SAFE: No JSON escaping needed
    prompt = current_prompt_template.replace("{review_text}", review_text)

    response = model.generate_content(prompt)

    completion_text = response.text.strip()

    json_start = completion_text.find("{")
    json_end = completion_text.rfind("}")
    if json_start != -1 and json_end != -1:
        json_str = completion_text[json_start:json_end+1]
    else:
        json_str = completion_text

    predicted = None
    explanation = ""

    try:
        parsed = json.loads(json_str)
        predicted = int(parsed.get("predicted_stars", None))
        explanation = parsed.get("explanation", "")
    except Exception as e:
        print("\nJSON Parse Error at index", idx)
        print("Raw Output:", completion_text)

    all_predictions.append(predicted)
    all_actual.append(actual_star)

    if idx < 3:
        print("\n======================")
        print("Review:", review_text[:200], "...")
        print("Model Output:", completion_text)
        print("JSON Extracted:", json_str)
        print("Predicted:", predicted, "| Actual:", actual_star)
        print("Explanation:", explanation)


  0%|          | 0/200 [00:00<?, ?it/s]


Review: We got here around midnight last Friday... the place was dead. However, they were still serving food and we enjoyed some well made pub grub. Service was friendly, quality cocktails were served, and th ...
Model Output: ```json
{
  "predicted_stars": 4,
  "explanation": "The review mentions enjoying well-made pub grub and friendly service. The reviewer notes the atmosphere is suitable for a sports bar and mentions it's a good spot for happy hour and day-drinking, indicating a positive experience overall."
}
```
JSON Extracted: {
  "predicted_stars": 4,
  "explanation": "The review mentions enjoying well-made pub grub and friendly service. The reviewer notes the atmosphere is suitable for a sports bar and mentions it's a good spot for happy hour and day-drinking, indicating a positive experience overall."
}
Predicted: 4 | Actual: 4
Explanation: The review mentions enjoying well-made pub grub and friendly service. The reviewer notes the atmosphere is suitable for a sports bar and

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15, model: gemini-2.0-flash
Please retry in 48.587959948s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, retry_delay {
  seconds: 48
}
]

In [218]:
correct = 0
valid = 0

for p, a in zip(all_predictions, all_actual):
    if p is not None:
        valid += 1
        if p == a:
            correct += 1

accuracy = correct / valid if valid > 0 else 0

print("\n===============================")
print("Valid Predictions:", valid)
print("Correct Predictions:", correct)
print("Accuracy:", accuracy)


Valid Predictions: 15
Correct Predictions: 11
Accuracy: 0.7333333333333333
